# Link Prediction with FastRP Embeddings
## TigerGraph ML Team

We will use FastRP embeddings of vertices to predict if there is a REVIEW edge between Users and Books.

## Connect to Database

In [ ]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(host="http://104.155.129.195", graphname="Amazon_OGB_Test")
embeddingDim = 128

## Gather Data
Here, we run a query to get both user and book embeddings for all edges in the graph. We have to batch the query because the REST endpoint and timeout limits.

In [ ]:
params = {
    "groups": 128,
    "groupId": 0,
}

users = []
books = []
ratings = []

for i in range(0, 70):
    print(i)
    params["groupId"] = i
    res = conn.runInstalledQuery("get_data", params=params)
    users += res[0]["@@userEmbeddings"]
    books += res[0]["@@bookEmbeddings"]
    ratings += res[0]["@@ratings"]


## Create Negative Samples
In order to have a proper dataset for classification, we need to generate negative samples. Negative samples are datapoints that are combinations of book and user embeddings that are not valid edges in the graph. To do this, we generate random shuffles of the book and user embeddings.

In [ ]:
import numpy as np
import random as rd
positiveSamples = np.append(np.array(res[0]["@@userEmbeddings"]), np.array(res[0]["@@bookEmbeddings"]), axis=1)
negativeSamples = np.append(np.array(rd.sample(res[0]["@@userEmbeddings"], k=len(res[0]["@@userEmbeddings"]))), np.array(rd.sample(res[0]["@@bookEmbeddings"], k=len(res[0]["@@bookEmbeddings"]))), axis=1)

In [ ]:
positiveSamples.shape

In [ ]:
negativeSamples.shape

## Finish Dataset Prep
We have a dataset that now contains both positive and negative samples. First, create the labels for the dataset. Finally, we can split the dataset into train and test sets.

In [ ]:
y = np.append(np.ones(positiveSamples.shape[0]), np.zeros(negativeSamples.shape[0]))
X = np.append(positiveSamples, negativeSamples, axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Perform Logistic Regression Classification
We can now perform logistic regression classification on the dataset. This doesn't perform that well.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

## Look at PCA representation
We can now look at the PCA representation of the dataset. Notably, the classes are not linearly seperable. Maybe a neural net would perform better?

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
X_red = pca.fit_transform(X)

import matplotlib.pyplot as plt
plot = plt.scatter(X_red[:,0], X_red[:,1], c=y)
plt.legend(handles=plot.legend_elements()[0], labels=["Edge", "No Edge"])
plt.show()

In [ ]:
pca = PCA(n_components=3)
X_red = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection='3d')
ax.scatter(X_red[:,0], X_red[:,1], X_red[:,2], c=y)
ax.legend(handles=plot.legend_elements()[0], labels=["Edge", "No Edge"])
plt.show

## Neural Network
We can now use a neural network to perform classification. This performs **way** better than logistic regression.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(256, 512, 5), random_state=1)

clf.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))